In [24]:
# Librerías
!pip install pmdarima
!pip install skforecast
!pip install awswrangler
!pip install holidays


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [25]:
# ======================================================================================
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import pickle
import time
import os
from io import StringIO
import boto3
import contextlib
import re
import matplotlib.pyplot as plt
import awswrangler as wr
import holidays

plt.style.use('seaborn-darkgrid')

from io import BytesIO
import joblib

# pmdarima
from pmdarima import ARIMA
from pmdarima import auto_arima

# statsmodels
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose

# skforecast
from skforecast.Sarimax import Sarimax
from skforecast.ForecasterSarimax import ForecasterSarimax
from skforecast.model_selection_sarimax import backtesting_sarimax
from skforecast.model_selection_sarimax import grid_search_sarimax
from sklearn.metrics import mean_absolute_error

import warnings

### Reading ABT

In [26]:
# Set up S3 client
client = boto3.client('s3')
bucket_name = 'viamericas-datalake-dev-us-east-1-283731589572-analytics'
ABTversion = 'v3_update'
path = f'ABT{ABTversion}/' # Folder under analysis

In [27]:
##Conexion a ABT##
# Listing objects 
elements = client.list_objects(Bucket=bucket_name, Prefix=path)
# Initialize an empty list 
dfs = []
# Iterate through each CSV in the S3 path
for obj in elements.get('Contents', []):
#    print(obj)
    # Check if the object is a CSV file
    if obj['Key'].endswith('.csv'):
        # Read CSV content from S3
        response = client.get_object(Bucket=bucket_name, Key=obj['Key'])
        print(obj['Key'])
        csv_content = response['Body'].read().decode('utf-8')

        # Transform CSV content to df
        df = pd.read_csv(StringIO(csv_content))

        dfs.append(df)
final_df = pd.concat(dfs, ignore_index=True)

ABTv3_update/ABTv3_update.csv


In [28]:
#Transformo fecha
final_df['date'] = pd.to_datetime(final_df['date']).dt.date

In [29]:
final_df.date.max()
# Acoto fechas
#final_df = final_df[final_df['date'] < pd.to_datetime('2023-08-05').date()]

datetime.date(2024, 2, 3)

In [30]:
final_df.payer_country.unique()

array(['ARGENPER_ARGENTINA', 'MERCHANTRADE_BANGLADESH',
       'MUTHOOT-BANGLADESH_BANGLADESH', 'TRANSPAY_BANGLADESH',
       'TRANSFERTO - THUNES_BELGIUM', 'KORI_BENIN', 'TRANSPAY_BENIN',
       'ARGENPER_BOLIVIA', 'BANCO BISA_BOLIVIA',
       'BANCO GANADERO (BOLIVIA)_BOLIVIA', 'BANCO DAYCOVAL_BRAZIL',
       'BANCO RENDIMENTO_BRAZIL', 'MSBB MONEY_BRAZIL',
       'KORI_BURKINA FASO', 'NAFA_BURKINA FASO', 'TRANSPAY_CAMEROON',
       'ZEEPAY_CAMEROON', 'AFEX_CHILE', 'ARGENPER_CHILE',
       'BANCOLOMBIA_COLOMBIA', 'DAVIVIENDA_COLOMBIA',
       'GRUPO EXITO (TN)_COLOMBIA', 'TELEDOLAR (TN)_COSTA RICA',
       "NAFA_COTE D'IVOIRE (IVORY COAST)",
       "ZEEPAY_COTE D'IVOIRE (IVORY COAST)",
       'BANCO BHD LEON_DOMINICAN REPUBLIC',
       'CARIBE EXPRESS_DOMINICAN REPUBLIC', 'BANCO DE GUAYAQUIL_ECUADOR',
       'BANCO DEL AUSTRO_ECUADOR', 'BANCO PICHINCHA (TN)_ECUADOR',
       'EASY PAGOS_ECUADOR', 'ECUAGIROS_ECUADOR', 'GLOBAL ENVIOS_ECUADOR',
       'ABANK (TN)_EL SALVADOR', 'BANCO AGRI

In [31]:
final_df.payer_country.nunique()

133

In [32]:
## WE ADD DAY OF THE DEAD##

In [33]:
def day_of_the_dead(date, payer_country):
    if date.month == 11 and date.day == 2 and (payer_country == "ELEKTRA (MEXICO)_MEXICO" or payer_country == "BANCOPPEL (APPRIZA)_MEXICO"):
        return 1
    else:
        return 0

In [34]:
final_df['day_of_the_dead'] = final_df.apply(lambda x: day_of_the_dead(x['date'], x['payer_country']), axis=1)

In [35]:
def thanksgiving_day(date, payer_country):
    if date.month == 11 and date.day == 23 and (payer_country == "ELEKTRA (MEXICO)_MEXICO" or payer_country == "BANCOPPEL (APPRIZA)_MEXICO"):
        return 1
    else:
        return 0

In [36]:
final_df['thanksgiving_day'] = final_df.apply(lambda x: day_of_the_dead(x['date'], x['payer_country']), axis=1)

### Looping into payer list

In [37]:
payer_list = [ 
    #"ELEKTRA (MEXICO)_MEXICO",
    #"BANCO ATLANTIDA_HONDURAS", 
   # "G Y T CONTINENTAL_GUATEMALA", 
    #"BANCO RENDIMENTO_BRAZIL",
    #"CARIBE EXPRESS_DOMINICAN REPUBLIC",
   #"BANCO DE OCCIDENTE_HONDURAS",
     ]

In [38]:
for payer in payer_list:
   
    #### SETEO EL PAYER ####
    datos=final_df.loc[final_df.payer_country==payer]
    datos = datos.set_index('date')
    datos = datos.asfreq('D')
    
    datos = datos[['amount', 'var_rate_lag_2','var_rate_lag_3','var_rate_lag_4', 'var_rate_lag_5',
               'var_rate_lag_6','var_rate_lag_7','var_rate_lag_8','var_rate_lag_9',
               'var_rate_lag_10','var_rate_lag_11','var_rate_lag_12','var_rate_lag_13', 
               'var_rate_lag_14','var_rate_lag_15','var_rate_lag_16','var_rate_lag_17',
               'var_rate_lag_18','var_rate_lag_19','var_rate_lag_20','var_rate_lag_21',
               'var_rate_lag_22','var_rate_lag_23','var_rate_lag_24','var_rate_lag_25',
               'var_rate_lag_26','var_rate_lag_27','var_rate_lag_28','var_rate_lag_29', 
               'ratio_coupon_tx_lag_2', 
               'ratio_coupon_tx_lag_3', 'ratio_coupon_tx_lag_4', 'ratio_coupon_tx_lag_5', 
               'ratio_coupon_tx_lag_6', 'ratio_coupon_tx_lag_7', 'ratio_coupon_tx_lag_8', 
               'ratio_coupon_tx_lag_9', 'ratio_coupon_tx_lag_10', 'ratio_coupon_tx_lag_11', 
               'ratio_coupon_tx_lag_12', 'ratio_coupon_tx_lag_13', 'ratio_coupon_tx_lag_14',
               'ratio_coupon_tx_lag_15','ratio_coupon_tx_lag_16','ratio_coupon_tx_lag_17', 
               'ratio_coupon_tx_lag_18','ratio_coupon_tx_lag_19','ratio_coupon_tx_lag_20',
               'ratio_coupon_tx_lag_21','ratio_coupon_tx_lag_22','ratio_coupon_tx_lag_23',
               'ratio_coupon_tx_lag_24','ratio_coupon_tx_lag_25','ratio_coupon_tx_lag_26',
               'ratio_coupon_tx_lag_27','ratio_coupon_tx_lag_28','ratio_coupon_tx_lag_29',
               'tx_lag_2', 'tx_lag_3','tx_lag_4', 'tx_lag_5','tx_lag_6', 'tx_lag_7', 
               'tx_lag_8', 'tx_lag_9', 'tx_lag_10', 'tx_lag_11','tx_lag_12', 'tx_lag_13',
               'tx_lag_14', 'tx_lag_15', 'tx_lag_16','tx_lag_17', 'tx_lag_18','tx_lag_19',
               'tx_lag_20', 'tx_lag_21','var_tx_cancelled_lag_2', 'var_tx_cancelled_lag_3', 
               'var_tx_cancelled_lag_4', 'var_tx_cancelled_lag_5', 'var_tx_cancelled_lag_6', 
               'var_tx_cancelled_lag_7', 'var_tx_cancelled_lag_8', 'var_tx_cancelled_lag_9', 
               'var_tx_cancelled_lag_10', 'var_tx_cancelled_lag_11', 'var_tx_cancelled_lag_12', 
               'var_tx_cancelled_lag_13', 'var_tx_cancelled_lag_14','var_tx_cancelled_lag_15', 
               'var_tx_cancelled_lag_16', 'var_tx_cancelled_lag_17', 'var_tx_cancelled_lag_18',
               'var_tx_cancelled_lag_19', 'var_tx_cancelled_lag_20','var_tx_cancelled_lag_21', 
               'var_tx_cancelled_lag_22','var_tx_cancelled_lag_23', 'var_tx_cancelled_lag_24',
               'var_tx_cancelled_lag_25', 'var_tx_cancelled_lag_26','var_tx_cancelled_lag_27', 
               'var_tx_cancelled_lag_28', 'var_tx_cancelled_lag_29',
               'margin_lag_10', 'margin_lag_2', 'margin_lag_3', 'margin_lag_4', 'margin_lag_5',
               'margin_lag_6', 'margin_lag_7', 'margin_lag_8', 'margin_lag_9',
                 'is_holiday',
                 'is_fourth_of_july',
                 'var_30ds',
                  'christmas_day',
                  'new_year_day',
                 'thanksgiving_day',
                  'post_holiday',
                  'day_of_the_dead'
              ]]
    
    # Train-validation-test
    # ======================================================================================
    fin_train = '2023-03-31'
    fin_val = '2023-06-21'
    print(
        f"Fechas entrenamiento : {datos.index.min()} --- {datos.loc[:fin_train].index.max()}  "
        f"(n={len(datos.loc[:fin_train])})"
    )
    print(
        f"Fechas validacion    : {datos.loc[fin_train:].index.min()} --- {datos.loc[:fin_val].index.max()}  "
        f"(n={len(datos.loc[fin_train:fin_val])})"
    )
    print(
        f"Fechas test          : {datos.loc[fin_val:].index.min()} --- {datos.index.max()}  "
        f"(n={len(datos.loc[fin_val:])})"
    )

    exogenas=['var_rate_lag_2','var_rate_lag_3','var_rate_lag_4', 'var_rate_lag_5',
               'var_rate_lag_6','var_rate_lag_7','var_rate_lag_8','var_rate_lag_9',
               'var_rate_lag_10','var_rate_lag_11','var_rate_lag_12','var_rate_lag_13', 
               'var_rate_lag_14','var_rate_lag_15','var_rate_lag_16','var_rate_lag_17',
               'var_rate_lag_18','var_rate_lag_19','var_rate_lag_20','var_rate_lag_21',
               'var_rate_lag_22','var_rate_lag_23','var_rate_lag_24','var_rate_lag_25',
               'var_rate_lag_26','var_rate_lag_27','var_rate_lag_28','var_rate_lag_29', 
               'ratio_coupon_tx_lag_2', 
               'ratio_coupon_tx_lag_3', 'ratio_coupon_tx_lag_4', 'ratio_coupon_tx_lag_5', 
               'ratio_coupon_tx_lag_6', 'ratio_coupon_tx_lag_7', 'ratio_coupon_tx_lag_8', 
               'ratio_coupon_tx_lag_9', 'ratio_coupon_tx_lag_10', 'ratio_coupon_tx_lag_11', 
               'ratio_coupon_tx_lag_12', 'ratio_coupon_tx_lag_13', 'ratio_coupon_tx_lag_14',
               'ratio_coupon_tx_lag_15','ratio_coupon_tx_lag_16','ratio_coupon_tx_lag_17', 
               'ratio_coupon_tx_lag_18','ratio_coupon_tx_lag_19','ratio_coupon_tx_lag_20',
               'ratio_coupon_tx_lag_21','ratio_coupon_tx_lag_22','ratio_coupon_tx_lag_23',
               'ratio_coupon_tx_lag_24','ratio_coupon_tx_lag_25','ratio_coupon_tx_lag_26',
               'ratio_coupon_tx_lag_27','ratio_coupon_tx_lag_28','ratio_coupon_tx_lag_29',
               'tx_lag_2', 'tx_lag_3','tx_lag_4', 'tx_lag_5','tx_lag_6', 'tx_lag_7', 
               'tx_lag_8', 'tx_lag_9', 'tx_lag_10', 'tx_lag_11','tx_lag_12', 'tx_lag_13',
               'tx_lag_14', 'tx_lag_15', 'tx_lag_16','tx_lag_17', 'tx_lag_18','tx_lag_19',
               'tx_lag_20', 'tx_lag_21','var_tx_cancelled_lag_2', 'var_tx_cancelled_lag_3', 
               'var_tx_cancelled_lag_4', 'var_tx_cancelled_lag_5', 'var_tx_cancelled_lag_6', 
               'var_tx_cancelled_lag_7', 'var_tx_cancelled_lag_8', 'var_tx_cancelled_lag_9', 
               'var_tx_cancelled_lag_10', 'var_tx_cancelled_lag_11', 'var_tx_cancelled_lag_12', 
               'var_tx_cancelled_lag_13', 'var_tx_cancelled_lag_14','var_tx_cancelled_lag_15', 
               'var_tx_cancelled_lag_16', 'var_tx_cancelled_lag_17', 'var_tx_cancelled_lag_18',
               'var_tx_cancelled_lag_19', 'var_tx_cancelled_lag_20','var_tx_cancelled_lag_21', 
               'var_tx_cancelled_lag_22','var_tx_cancelled_lag_23', 'var_tx_cancelled_lag_24',
               'var_tx_cancelled_lag_25', 'var_tx_cancelled_lag_26','var_tx_cancelled_lag_27', 
               'var_tx_cancelled_lag_28', 'var_tx_cancelled_lag_29',
               'margin_lag_10', 'margin_lag_2', 'margin_lag_3', 'margin_lag_4', 'margin_lag_5',
               'margin_lag_6', 'margin_lag_7', 'margin_lag_8', 'margin_lag_9',
                 'is_holiday',
                 'is_fourth_of_july',
                 'var_30ds',
                  'christmas_day',
                  'new_year_day',
                'thanksgiving_day',
                  'post_holiday',
                  'day_of_the_dead'
              ]
    
    #Filling NaN
    datos[exogenas] = datos[exogenas].fillna(0)
    
    # Modelo SARIMAX con skforecast.Sarimax
    # ==============================================================================
    warnings.filterwarnings("ignore", category=UserWarning, message='Non-invertible|Non-stationary')
    modelo = Sarimax(order=(2, 1, 1), seasonal_order=(2, 1, 1, 7))
    modelo.fit(y=datos['amount'].loc[:fin_train], exog=datos[exogenas].loc[:fin_train] )
    modelo.summary()
    #warnings.filterwarnings("default")
    
    #Guardo los resultados de Sarimax en un txt
    file_name = f'sarimax_summary_{payer}_{ABTversion}.txt'
    
    with open(file_name, 'w') as f:
        f.write(modelo.summary().as_text())

        
    # Se filtran variables exógenas significativas
    # Obtener la tabla de resumen como cadena
    tabla_resumen = modelo.summary().tables[1].as_html()
    # Convertir la cadena HTML en un DataFrame de pandas
    df_resumen = pd.read_html(tabla_resumen, header=0, index_col=0)[0]
    # Filtrar el DataFrame para quedarte solo con las filas que contienen valores p
    df_valores_p = df_resumen[df_resumen.index.str.contains('P>|z|')]
    # Definir el umbral de significancia
    umbral_significancia = 0.05
    # Filtrar las filas donde el valor p sea menor que el umbral de significancia
    variables_significativas = df_valores_p[df_valores_p['P>|z|'] < umbral_significancia]
    indices_variables_significativas = variables_significativas.index.tolist()
    # Convertir los índices en una lista de caracteres
    exogenas_total = list(map(str, indices_variables_significativas))
    # Lista de exogenas filtrada
    exogenas_filtered = [var for var in exogenas_total if not var.startswith(('ar.', 'ma.', 'sigma'))]
    # Mostrar la lista de exogenas filtrada
    print('umbral de significancia: 0.05 - ', exogenas_filtered)
    
    #exogenas_filtered = ['var_rate_lag_8', 'var_rate_lag_9', 'var_rate_lag_10', 'var_rate_lag_11', 'var_rate_lag_12', 'var_rate_lag_13', 'var_rate_lag_14', 'var_rate_lag_15', 'var_rate_lag_16', 'var_rate_lag_17', 'var_rate_lag_18', 'var_rate_lag_19', 'ratio_coupon_tx_lag_2', 'ratio_coupon_tx_lag_7', 'ratio_coupon_tx_lag_8', 'ratio_coupon_tx_lag_9', 'ratio_coupon_tx_lag_15', 'ratio_coupon_tx_lag_16', 'ratio_coupon_tx_lag_22', 'tx_lag_2', 'tx_lag_3', 'tx_lag_5', 'tx_lag_6', 'tx_lag_7', 'tx_lag_10', 'tx_lag_13', 'tx_lag_14', 'var_30ds', 'christmas_day', 'new_year_day', 'post_holiday']
    # Abro el archivo en modo escritura, con la opción 'append' para agregar contenido al final del archivo
    with open(file_name, 'a') as f:
        # Escribir el contenido de la variable 'exogenas' en el archivo
        f.write('\n\nContenido de la variable "exogenas":\n')
        f.write(str(exogenas_filtered))
        
    # Unifico los listados
#    variables = exogenas_filtered + dummies
    
        
    # Auto arima: seleccion basada en AIC
    # ==============================================================================
    resultados_autoarima = auto_arima(
                y                 = datos['amount'].loc[:fin_val],
                exog              = datos[exogenas_filtered].loc[:fin_val],
                start_p           = 0,
                start_q           = 0,
                max_p             = 7,
                max_q             = 7,
                seasonal          = True,
                #test              = 'adf',
                m                 = 7, # periodicidad de la estacionalidad
                #d                 = 1, # El algoritmo determina 'd' (lo obligo porque claramente es no estacionario)
                #D                 = 1, # El algoritmo determina 'D' (idem arriba)
                max_d             = 2,
                max_D             = 2,
                trace             = True,
                error_action      = 'ignore',
                suppress_warnings = True,
                stepwise          = True
            )
    opt_order_2 = resultados_autoarima.order
    opt_seasonal_order_2 = resultados_autoarima.seasonal_order
    
    forecaster_autoarima = ForecasterSarimax(
                     regressor=Sarimax(order=opt_order_2, seasonal_order=opt_seasonal_order_2)
                 )
    metrica_autoarima, predicciones_autoarima = backtesting_sarimax(
                                    forecaster            = forecaster_autoarima,
                                    y                     = datos['amount'],
                                    exog                  = datos[exogenas_filtered],
                                    initial_train_size    = len(datos.loc[:fin_val]),
                                    steps                 = 2,
                                    metric                = 'mean_absolute_percentage_error',
                                    refit                 = False,
                                    n_jobs                = 'auto',
                                    suppress_warnings_fit = True,
                                    verbose               = False,
                                    show_progress         = True
                                )
    # Predicción
    nueva_fecha = datetime.strptime(fin_val, '%Y-%m-%d') + timedelta(days=1)
    predicciones_autoarima = predicciones_autoarima['pred']

    # Combinar los dos DataFrames en uno
    resultados_predichos_autoarima= pd.DataFrame({'date': predicciones_autoarima.index, 'valor_real': datos['amount'].loc[nueva_fecha:], 'valor_predicho': predicciones_autoarima})
    # Calcular el MAPE
    resultados_predichos_autoarima['mape'] = abs((resultados_predichos_autoarima['valor_real'] - resultados_predichos_autoarima['valor_predicho']) / resultados_predichos_autoarima['valor_real']) * 100
    resultados_predichos_autoarima['error_abs'] = abs((resultados_predichos_autoarima['valor_real'] - resultados_predichos_autoarima['valor_predicho']))
    # Calcular el promedio de la columna 'mape'
    promedio_mape_autoarima = resultados_predichos_autoarima['mape'].mean()
    promedio_error_abs_autoarima = resultados_predichos_autoarima['error_abs'].mean()
    # Imprimir el promedio
    print("Promedio de la columna 'mape' - 2 dias:", promedio_mape_autoarima)
    print("Promedio de la columna 'error_abs' - 2 dias:", promedio_error_abs_autoarima)
    
    
    ### FIT
    forecaster_autoarima.fit(y=datos['amount'],exog=datos[exogenas_filtered] ,suppress_warnings=True)
    
    # Crear una carpeta llamada 'resultados' si no existe
    #folder_name = 'Last_releases/'
    folder_name = 'top_payers_abt_v3update/'
    carpeta_resultados = path+folder_name+payer+'/'
    print(carpeta_resultados)

    try:
        client.put_object(Bucket=bucket_name, Key=(carpeta_resultados+'/'))
        print(f"Carpeta '{carpeta_resultados}' creada en el bucket '{bucket_name}'.")
    except Exception as e:
        print(f"Error al crear la carpeta: {e}")
        
    #GUARDO RESULTADOS EN EXCEL
    nombre_archivo_excel = os.path.join(carpeta_resultados, 'predicciones_test_2d_'+payer+'.xlsx')

    # Guarda el DataFrame en un archivo XLSX en S3 usando AWS Data Wrangler
    wr.s3.to_excel(resultados_predichos_autoarima, f"s3://{bucket_name}/{nombre_archivo_excel}", index=False)
    nombre_archivo_pickle = os.path.join(carpeta_resultados, 'MODEL_2d_'+payer+'.pkl')
    
    
    # Guarda el modelo en un archivo pickle temporal
    modelo_temp_file = 'MODEL_2d_'+payer+'.pkl'
    joblib.dump(forecaster_autoarima, modelo_temp_file)
    # Sube el archivo pkl a S3
    try:
        wr.s3.upload(modelo_temp_file, f"s3://{bucket_name}/{nombre_archivo_pickle}")
        print(f"Modelo guardado como pkl en S3: {nombre_archivo_pickle}")
        os.remove(modelo_temp_file) # Elimina el temporal
    except Exception as e:
        print(f"Error al subir el modelo a S3: {e}")
        
    # Abro el archivo en modo escritura, con la opción 'append' para agregar contenido al final del archivo
    with open(file_name, 'a') as f:
        # Escribir el contenido de la variable 'exogenas' en el archivo
        f.write('\n\nModelo Seleccionado:\n')
        f.write(str(forecaster_autoarima))
        f.write('\n\nMAPE 2 dias:\n')
        f.write(str(promedio_mape_autoarima))

#     ### PREDICCION A 7 DIAS    
#     metrica_autoarima, predicciones_autoarima = backtesting_sarimax(
#                                 forecaster            = forecaster_autoarima,
#                                 y                     = datos['amount'],
#                                 exog                  = datos[exogenas_filtered],
#                                 initial_train_size    = len(datos.loc[:fin_val]),
#                                 steps                 = 7,
#                                 metric                = 'mean_absolute_percentage_error',
#                                 refit                 = False,
#                                 n_jobs                = 'auto',
#                                 suppress_warnings_fit = True,
#                                 verbose               = False,
#                                 show_progress         = True
#                             )
#     # Predicción
#     nueva_fecha = datetime.strptime(fin_val, '%Y-%m-%d') + timedelta(days=1)
#     predicciones_autoarima = predicciones_autoarima['pred']

#     # Combinar los dos DataFrames en uno
#     resultados_predichos_autoarima_7d= pd.DataFrame({'date': predicciones_autoarima.index, 'valor_real': datos['amount'].loc[nueva_fecha:], 'valor_predicho': predicciones_autoarima})
#     # Calcular el MAPE
#     resultados_predichos_autoarima_7d['mape'] = abs((resultados_predichos_autoarima_7d['valor_real'] - resultados_predichos_autoarima_7d['valor_predicho']) / resultados_predichos_autoarima_7d['valor_real']) * 100
#     resultados_predichos_autoarima_7d['error_abs'] = abs((resultados_predichos_autoarima_7d['valor_real'] - resultados_predichos_autoarima_7d['valor_predicho']))
#     # Calcular el promedio de la columna 'mape'
#     promedio_mape_autoarima_7d = resultados_predichos_autoarima_7d['mape'].mean()
#     promedio_error_abs_autoarima_7d = resultados_predichos_autoarima_7d['error_abs'].mean()
#     # Imprimir el promedio
#     print("Promedio de la columna 'mape' - 7 dias:", promedio_mape_autoarima_7d)
#     print("Promedio de la columna 'error_abs - 7 dias':", promedio_error_abs_autoarima_7d)
    
    
#     # #GUARDO RESULTADOS EN EXCEL
#     # nombre_archivo_excel = os.path.join(carpeta_resultados, 'predicciones_test_7d_'+payer+'.xlsx')
#     # # Guarda el DataFrame en un archivo XLSX en S3 usando AWS Data Wrangler
#     # wr.s3.to_excel(resultados_predichos_autoarima_7d, f"s3://{bucket_name}/{nombre_archivo_excel}", index=False)
    
#     with open(file_name, 'a') as f:
#         f.write('\n\nMAPE 7 dias:\n')
#         f.write(str(promedio_error_abs_autoarima_7d))
    
    #TXT
    ruta_txt = os.path.join(carpeta_resultados, file_name)
    # Cargar el archivo en S3
    s3 = boto3.resource("s3")
    s3.Bucket(bucket_name).upload_file(file_name, ruta_txt)
    os.remove(file_name) # Elimina el temporal

Fechas entrenamiento : 2021-01-01 00:00:00 --- 2023-03-31 00:00:00  (n=820)
Fechas validacion    : 2023-03-31 00:00:00 --- 2023-06-21 00:00:00  (n=83)
Fechas test          : 2023-06-21 00:00:00 --- 2024-02-03 00:00:00  (n=228)


/opt/conda/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/tmp/ipykernel_18/3618330673.py:131: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_resumen = pd.read_html(tabla_resumen, header=0, index_col=0)[0]


umbral de significancia: 0.05 -  ['var_rate_lag_7', 'var_rate_lag_14', 'var_rate_lag_15', 'var_rate_lag_21', 'var_rate_lag_24', 'ratio_coupon_tx_lag_2', 'ratio_coupon_tx_lag_3', 'ratio_coupon_tx_lag_4', 'ratio_coupon_tx_lag_5', 'ratio_coupon_tx_lag_6', 'ratio_coupon_tx_lag_7', 'ratio_coupon_tx_lag_8', 'ratio_coupon_tx_lag_9', 'ratio_coupon_tx_lag_10', 'ratio_coupon_tx_lag_11', 'ratio_coupon_tx_lag_12', 'ratio_coupon_tx_lag_13', 'ratio_coupon_tx_lag_14', 'ratio_coupon_tx_lag_15', 'ratio_coupon_tx_lag_16', 'ratio_coupon_tx_lag_17', 'ratio_coupon_tx_lag_18', 'ratio_coupon_tx_lag_19', 'ratio_coupon_tx_lag_21', 'ratio_coupon_tx_lag_22', 'ratio_coupon_tx_lag_23', 'ratio_coupon_tx_lag_24', 'ratio_coupon_tx_lag_26', 'ratio_coupon_tx_lag_28', 'ratio_coupon_tx_lag_29', 'tx_lag_2', 'tx_lag_3', 'tx_lag_4', 'tx_lag_5', 'tx_lag_7', 'tx_lag_8', 'tx_lag_9', 'tx_lag_10', 'tx_lag_11', 'tx_lag_16', 'is_fourth_of_july', 'var_30ds', 'christmas_day', 'new_year_day', 'post_holiday']
Performing stepwise searc

  0%|          | 0/114 [00:00<?, ?it/s]

Promedio de la columna 'mape' - 2 dias: 8.548615926566757
Promedio de la columna 'error_abs' - 2 dias: 651367.0225418437
ABTv3_update/top_payers_abt_v3update/ELEKTRA (MEXICO)_MEXICO/
Carpeta 'ABTv3_update/top_payers_abt_v3update/ELEKTRA (MEXICO)_MEXICO/' creada en el bucket 'viamericas-datalake-dev-us-east-1-283731589572-analytics'.
Modelo guardado como pkl en S3: ABTv3_update/top_payers_abt_v3update/ELEKTRA (MEXICO)_MEXICO/MODEL_2d_ELEKTRA (MEXICO)_MEXICO.pkl


In [39]:
ruta_txt

'ABTv3_update/top_payers_abt_v3update/ELEKTRA (MEXICO)_MEXICO/sarimax_summary_ELEKTRA (MEXICO)_MEXICO_v3_update.txt'

In [40]:
nombre_archivo_excel

'ABTv3_update/top_payers_abt_v3update/ELEKTRA (MEXICO)_MEXICO/predicciones_test_2d_ELEKTRA (MEXICO)_MEXICO.xlsx'

In [41]:
forecaster_autoarima

ForecasterSarimax 
Regressor: Sarimax(6,1,1)(1,0,1)[7] 
Regressor parameters: {'concentrate_scale': False, 'dates': None, 'disp': False, 'enforce_invertibility': True, 'enforce_stationarity': True, 'freq': None, 'hamilton_representation': False, 'maxiter': 50, 'measurement_error': False, 'method': 'lbfgs', 'missing': 'none', 'mle_regression': True, 'order': (6, 1, 1), 'seasonal_order': (1, 0, 1, 7), 'simple_differencing': False, 'sm_fit_kwargs': {}, 'sm_init_kwargs': {}, 'sm_predict_kwargs': {}, 'start_params': None, 'time_varying_regression': False, 'trend': None, 'trend_offset': 1, 'use_exact_diffuse': False, 'validate_specification': True} 
fit_kwargs: {} 
Window size: 1 
Transformer for y: None 
Transformer for exog: None 
Exogenous included: True 
Type of exogenous variable: <class 'pandas.core.frame.DataFrame'> 
Exogenous variables names: ['var_rate_lag_7', 'var_rate_lag_14', 'var_rate_lag_15', 'var_rate_lag_21', 'var_rate_lag_24', 'ratio_coupon_tx_lag_2', 'ratio_coupon_tx_lag_3'